In [205]:
import random as rnd
import string 
from tqdm import tqdm
import math
import mmh3

In [206]:
def gen_rnd_string(len):
    return "".join(rnd.choice(string.ascii_letters + string.digits) for _ in range(len))

In [213]:
def list_to_hash(bloom):
    hash = 0
    two_pow = 1
    for i in bloom:
        hash += two_pow * i
        two_pow *= 2

    return hash

генерируем полбзователей и их бакеты с адрессами и токены на которые указывают аддреса

In [214]:
t = 1000 # кол-во пользователей
r = 100 # кол-во аддресов в бакете

address = []
tokens = []
address_token_map = dict()

for i in tqdm(range(t)):
    # пока каждый user имеет только один бакет с адресами
    usesr_address = []
    users_token = []

    for j in range(r):
        cur_addres = int(rnd.uniform(1, 1_000_000_000))
        cur_token = gen_rnd_string(10)
        usesr_address.append(cur_addres)
        users_token.append(cur_token)
        address_token_map[cur_addres] = cur_token

    address.append(usesr_address)
    tokens.append(users_token)


100%|██████████| 1000/1000 [00:00<00:00, 1695.69it/s]


In [215]:
m = 5000
n = 1000
k = 22

blooms = []
blooms_token_map = dict()
address_bloom_map = dict()
token_bloom_map = dict()

for i in tqdm(range(t)):
    for o, cur_addres in enumerate(address[i]):
        cur_bloom = [0]*m
        cur_token = tokens[i][o]
        for j in range(k):
            nonce = f'nonce_{j}'
            index = mmh3.hash128(str(cur_addres) + nonce) % m
            cur_bloom[index] = 1
        blooms.append(cur_bloom)
        blooms_token_map[list_to_hash(cur_bloom)] = cur_token
        token_bloom_map[cur_token] = cur_bloom
        address_bloom_map[cur_addres] = cur_bloom

100%|██████████| 1000/1000 [01:40<00:00,  9.92it/s]


In [243]:
def find_l(N):
    return math.log(1 - (n/N)**(1/k))/math.log(1-1/m) - r*k

In [244]:
def gen_user_bloom(user_number, N):
    user_bloom = [0]*m

    for cur_addres in address[user_number]:
        cur_bloom = address_bloom_map[cur_addres]
        for ind, val in enumerate(cur_bloom):
            if val == 1:
                user_bloom[ind] = 1
    
    l = find_l(N)
    print('l = ' + str(int(l)))

    for i in range(int(l)):
        index = int(rnd.uniform(0, m))
        user_bloom[index] = 1

    print('user bloom is finished!')
    return user_bloom
    

In [245]:
def find(user_number, N):

    ans = []
    user_bloom = gen_user_bloom(user_number, N)

    for j in tqdm(range(N)):
        bloom = blooms[j]
        flag = True
        for i, val in enumerate(bloom):
            if val == 1 and user_bloom[i] == 0:
                flag = False
                break
        
        if flag:
            ans.append(blooms_token_map[list_to_hash(bloom)])

    print('user bloom:')
    cnt_1 = 0
    for i in user_bloom:
        cnt_1 += i
    print('percent of 1: ' + str(cnt_1/m) + '%')

    # print('tokens bloom:')
    # for token in tokens[user_number]:
    #     print(token_bloom_map[token])

    print('server find is finished')
    return ans

In [246]:
def check_true(user_number, N):
    server_ans = find(user_number, N)

    # print(tokens[user_number])
    # print(server_ans)

    print('count returned: ' + str(len(server_ans)))

    for token in tqdm(tokens[user_number]):
        if token in server_ans:
            continue
        else:
            return False

    
    return True

In [253]:
for i in [2, 10, 20, 50, 54]:
    res = check_true(i, 50000)
    print("algo is: " + str(res))

l = 6871
user bloom is finished!


100%|██████████| 50000/50000 [00:04<00:00, 10422.42it/s]


user bloom:
percent of 1: 0.838%
server find is finished
count returned: 1138


100%|██████████| 100/100 [00:00<00:00, 991561.23it/s]


algo is: True
l = 6871
user bloom is finished!


...

algo is: True
l = 6871
user bloom is finished!


100%|██████████| 50000/50000 [00:03<00:00, 13879.52it/s]


user bloom:
percent of 1: 0.8382%
server find is finished
count returned: 1114


100%|██████████| 100/100 [00:00<00:00, 440578.15it/s]

algo is: True
